In [ ]:
import numpy as np
from math import *
from numpy import sin, cos

# Constants
a = 3.905
tl = 875 *10**(-3)/2.
th = 40 *10**(-3)/2.
td = 40 *10**(-3)/2.
U = 2.7
dE =  47*10**(-3)
dso = 40*10**(-3)

def create_H(kxa, kya, N1, N2, N3):
    
    e1 = 2*tl*(2 - cos(kxa) - cos(kya))
    e2 = 2*tl*(1 - cos(kxa)) + 2*th*(1 - cos(kya))
    e3 = 2*th*(1 - cos(kxa)) + 2*tl*(1 - cos(kya))
    
    ### H0 Matrix
    H0_little = np.matrix([[e1-dE,           0,                        0               ],
                           [  0,             e2,               2*td*sin(kxa)*sin(kya)  ],
                           [  0,     2*td*sin(kxa)*sin(kya),           e3              ]])
    
    H0 = np.kron(H0_little, np.identity(2))
        
    ### Hso Matrix
    Hso = (1j*dso/2)*np.matrix([[ 0,  0,  0,  1,  0, 1j],
                                [ 0,  0,  1,  0,-1j,  0], 
                                [ 0, -1,  0,  0,  1,  0], 
                                [-1,  0,  0,  0,  0, -1], 
                                [ 0,-1j, -1,  0,  0,  0], 
                                [1j,  0,  0,  1,  0,  0]])
    
    ### Hint matrix
    E1 = 0.5*U*N1 +     U*N2 +     U*N3;
    E2 =     U*N1 + 0.5*U*N2 +     U*N3;
    E3 =     U*N1 +     U*N2 + 0.5*U*N3;
    
    Hint_little = np.matrix([[E1, 0,   0], 
                             [0,  E2,  0],
                             [0,   0, E3]])
    
    Hint = np.kron(Hint_little, np.identity(2))
    
    return H0 + Hso + Hint 

import numba
#@numba.jit(nopython=1)
def create_H_faster(kxa, kya, N1, N2, N3):
    
    # normal energies
    e1 = 2*tl*(2 - cos(kxa) - cos(kya))
    e2 = 2*tl*(1 - cos(kxa)) + 2*th*(1 - cos(kya))
    e3 = 2*th*(1 - cos(kxa)) + 2*tl*(1 - cos(kya))    
    eside = 2*td*sin(kxa)*sin(kya)
    
    ### Interaction erergies
    E1 = 0.5*U*N1 +     U*N2 +     U*N3;
    E2 =     U*N1 + 0.5*U*N2 +     U*N3;
    E3 =     U*N1 +     U*N2 + 0.5*U*N3;
    
    ### The whole H Matrix
    #H = np.zeros((6,6), dtype=numba.complex128)
    H = np.zeros((6,6), dtype=np.complex128)
    H[0,0]=e1-dE+E1 ;H[0,1]=    0     ;H[0,2]=       0       ;H[0,3]=  1j*dso/2    ;H[0,4]=     0        ;H[0,5]=   -dso/2
    H[1,0]=   0     ;H[1,1]=e1-dE + E1;H[1,2]=   1j*dso/2    ;H[1,3]=      0       ;H[1,4]=   dso/2      ;H[1,5]=      0    
    H[2,0]=   0     ;H[2,1]=-1j*dso/2 ;H[2,2]=    e2 + E2    ;H[2,3]=      0       ;H[2,4]=1j*dso/2+eside;H[2,5]=      0
    H[3,0]=-1j*dso/2;H[3,1]=    0     ;H[3,2]=       0       ;H[3,3]=   e2 + E2    ;H[3,4]=     0        ;H[3,5]=-1j*dso/2+eside
    H[4,0]=   0     ;H[4,1]=  dso/2   ;H[4,2]=-1j*dso/2+eside;H[4,3]=      0       ;H[4,4]=  e3 + E3     ;H[4,5]=      0
    H[5,0]= -dso/2  ;H[5,1]=    0     ;H[5,2]=       0       ;H[5,3]=1j*dso/2+eside;H[5,4]=     0        ;H[5,5]=   e3 + E3
        
    return H

In [ ]:
from scipy import integrate, optimize
from sympy.functions.special.delta_functions import Heaviside

integrand_counter = 0
energy_counter = 0
bound_counter = 0
statistics = []

def zero_statistics():
    global integrand_counter, energy_counter, bound_counter, statistics
    integrand_counter, energy_counter, bound_counter = 0, 0, 0
    statistics = []

def m_integrand(N1, N2, N3, mu):
    def integrand(kxa, kya, ms):
        global integrand_counter, statistics
        integrand_counter += 1
        statistics.append((kxa, kya))
        H = create_H_faster(kxa, kya, N1, N2, N3) 
        e, psi = np.linalg.eigh(H)
        
        # leave only the cols asked asked by m
        cols = np.array([(int(i/2) in ms) for i in range(6)])        
        psi = psi[:,cols]
        
        total = np.array([
                  np.sum(np.abs(psi[2*j:2*j+2,:])**2)
                  for j in xrange(3)
                ])
        
        return  (1/(2*pi)**2) * total
            
    return integrand

def m_energy(N1, N2, N3, mu):
    def energy(kxa, kya, ms):
        global energy_counter
        energy_counter += 1
        H = create_H_faster(kxa, kya, N1, N2, N3) 
        e = np.linalg.eigvalsh(H)
        return  [e[2*m] for m in ms]
    return energy

# this function assumes that the eig_values are monotoneous .
# @return for every kxa, the positive kya value that will match the fermi area
def m_fermi_area(N1, N2, N3, m, mu):
    def area(kxa):
        global bound_counter
        bound_counter += 1
        
        e_func = m_energy(N1, N2, N3, mu)
        fermi_func = lambda kya: e_func(kxa, kya, [m])[0] - mu 
        
        fa, fb = fermi_func(0), fermi_func(pi)
        
        # find the first kya value of eig_func that is above mu
        if np.sign(fa) == -1 and np.sign(fb) == -1:
            return pi
        elif np.sign(fa) == 1 and np.sign(fb) == 1:
            return 0
        else:
            return optimize.brentq(fermi_func, 0, pi)
    return area

integ = m_integrand(1, 1, 1, 0.95)
integ(0, 0, [0])

In [ ]:
import cProfile
import pstats

##### integrate slowly but accurately

def integrate_slower(N, mu):   
    # integrate over kxa, kya in the fermi area for every m
    total = [0]*3
    
    for j in range(3):
        total[j] += integrate.dblquad(
            lambda y, x: m_integrand(N[0], N[1], N[2], mu)(x, y, [0])[j], 
            0.0, pi, lambda x: 0.0, m_fermi_area(N[0], N[1], N[2], 0, mu),        
        )[0]
        total[j] += integrate.dblquad(
            lambda y, x: m_integrand(N[0], N[1], N[2], mu)(x, y, [1])[j], 
            0.0, pi, lambda x: 0.0, m_fermi_area(N[0], N[1], N[2], 1, mu),        
        )[0]
        total[j] += integrate.dblquad(
            lambda y, x: m_integrand(N[0], N[1], N[2], mu)(x, y, [2])[j], 
            0.0, pi, lambda x: 0.0, m_fermi_area(N[0], N[1], N[2], 2, mu),        
        )[0]
    
    return total
    
def integrate_slower(N, mu):   
    # integrate over kxa, kya in the fermi area for every m
    total = [0]*3
    
    for j in range(3):
        total[j] += integrate.dblquad(
            lambda y, x: m_integrand(N[0], N[1], N[2], mu)(x, y, [0, 1, 2])[j], 
            0.0, pi, lambda x: 0.0, m_fermi_area(N[0], N[1], N[2], 2, mu),        
        )[0]
        total[j] += integrate.dblquad(
            lambda y, x: m_integrand(N[0], N[1], N[2], mu)(x, y, [0, 1])[j], 
            0.0, pi, m_fermi_area(N[0], N[1], N[2], 2, mu), m_fermi_area(N[0], N[1], N[2], 1, mu),
        )[0]
        total[j] += integrate.dblquad(
            lambda y, x: m_integrand(N[0], N[1], N[2], mu)(x, y, [0])[j], 
            0.0, pi, m_fermi_area(N[0], N[1], N[2], 1, mu), m_fermi_area(N[0], N[1], N[2], 0, mu),
        )[0]
    
    return np.array(total)*4.

zero_statistics()
#print integrate_slower([guess_N1, guess_N2, guess_N3], mu)
#print "The integrand was called", integrand_counter, "times"
#s = cProfile.run("m_integrate([guess_N1, guess_N2, guess_N3], mu)", "prfiler_data")
#pstats.Stats("prfiler_data").sort_stats("tottime").print_stats()
#%prun m_integrate([guess_N1, guess_N2, guess_N3], mu)

In [ ]:
### integrate faster and more efficiently using self-implemented integration 

def my_integrate3(func, xa, xb, ya, yb1, yb2, yb3, yc, xiters):
    dx = dy = (xb-xa)/xiters
    totals = np.array([0]*3)
    for i in range(xiters):
        x = xa + i*dx
        ylims =  [(yb1(x),0), (yb2(x),1) , (yb3(x),2)]
        ylims = sorted(ylims, key=lambda a: a[0])
        ylims = [(ya, )] + ylims
        current_ms = [0, 1, 2]
        
        for i in range(1, 4):
            ystart = ylims[i-1][0]
            yend = ylims[i][0]
            yiters = int((yend-ystart)/dy)

            for j in range(yiters):
                y = ystart + j*dy
                totals = np.add(totals, func(x, y, current_ms)*dx*dy)
            current_ms.remove(ylims[i][1])
            
    return totals

zero_statistics()
def integrate_faster(N, mu):
    return np.array(my_integrate3(
        m_integrand(N[0], N[1], N[2], mu),
        0.0, pi, 
        0.0, 
        m_fermi_area(N[0], N[1], N[2], 0, mu),
        m_fermi_area(N[0], N[1], N[2], 1, mu),
        m_fermi_area(N[0], N[1], N[2], 2, mu),
        pi,
        1000
    )) * 4.

#import line_profiler
#print integrate_faster([guess_N1, guess_N2, guess_N3], mu)
#s = cProfile.run("print integrate_faster([guess_N1, guess_N2, guess_N3], mu)", "prfiler_data")
#pstats.Stats("prfiler_data").sort_stats("tottime").print_stats()
#%lprun integrate_faster([guess_N1, guess_N2, guess_N3], mu)
#print "The integrand was called", integrand_counter, "times"

In [ ]:
import functools
mu = 0.26

def global_params_str():
    global a,tl,th,td,U,dE,dso
    return "a=%f,tl=%f,th=%f,td=%f,U=%f,dE=%f,dso=%f" % (a,tl,th,td,U,dE,dso)

# note that this decorator ignores **kwargs
def memoize(obj):
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        print "Cache: ", str(args)
        g = global_params_str()
        if g not in cache:
            print "Globals not in cache. Creating new one"
            cache[g] = {}
        if str(args) not in cache[g]:
            print "Not in cache", cache[g]
            cache[g][str(args)] = obj(*args, **kwargs)
        return cache[g][str(args)]
    return memoizer

def zero_me_mu(mu):
    def zero_me(N_input):
        full_input = [N_input[0], N_input[1], N_input[1]]
        N_output = np.array(integrate_slower(full_input, mu)[0:2])
        print N_input, "->", N_output
        return N_input - N_output
    return zero_me

@memoize
def calc_N(mu):
    return optimize.fsolve(zero_me_mu(mu), [guess_N1, guess_N2], xtol=0.01, factor=100)

def calc_n(mu, N):
    ans = np.array([
        integrate.dblquad(
            lambda x, y: 1,
            0.0, pi, lambda x: 0.0, m_fermi_area(N[0], N[1], N[1], m, mu),        
        )[0]
        for m in range(3)
    ])
    return 4*ans/((2*pi)**2) / (a*10**(-8))**2

zero_statistics()
N = [guess_N1, guess_N2] = calc_N(mu)
guess_N3 = guess_N2
print "N is", N
print "n is", calc_n(mu, N)

In [ ]:
#### Plot integrands and energies
mu = 0.2
#N = calc_N(mu)
N = [0.00811131,  0.04666291,  0.0466629]

import matplotlib as mpl
from matplotlib import pyplot
extent = pi
X = np.arange(-extent, extent, 0.06)
Y = np.arange(-extent, extent, 0.06)

fig = mpl.pyplot.figure()
zero_statistics()

def draw_Z(sp, Z, area):
    sp.imshow(Z, extent=(-extent, extent, -extent, extent))
    sp.contour(Z, extent=(-extent, extent, -extent, extent))

    # add the line
    sp.plot(X, [ min(area(x), extent) for x in X], "k-")
    sp.plot(X, [-min(area(x), extent) for x in X], "k-")

for m in range(3):

    area = m_fermi_area(N[0], N[1], N[1], m, mu)
    energy = m_energy(N[0], N[1], N[1], mu)
    integrand = m_integrand(N[0], N[1], N[1], mu)
    
    # create the energy subplot
    Z = [[j for j in X] for i in Y]
    for ix in range(len(X)):
        y_lim = area(X[ix])
        for iy in range(len(Y)):
            Z[iy][ix] = energy(X[ix], Y[iy], [m])[0]

    energy_sp = fig.add_subplot(3, 4, 1+m*4)
    draw_Z(energy_sp, Z, area)
    if m == 0:
        energy_sp.set_title("Energy")
    
    # create the three graphs
    Zs = [np.zeros((len(X), len(Y))), np.zeros((len(X), len(Y))), np.zeros((len(X), len(Y)))]
    for ix in range(len(X)):
        y_lim = area(X[ix])
        for iy in range(len(Y)):
            Zs[0][iy][ix], Zs[1][iy][ix], Zs[2][iy][ix] = integrand(X[ix], Y[iy], [m])

    for j in range(3):
        integrand_sp = fig.add_subplot(3, 4, 1+m*4+1+j)
        draw_Z(integrand_sp, Zs[j], area)
        if m == 0:
            integrand_sp.set_title("integrand j=" + str(j))

        
mpl.pyplot.show(fig)

In [ ]:
#### view integrand

import matplotlib as mpl
from matplotlib import pyplot
X = np.arange(0, pi, 0.7)
Y = np.arange(0, pi, 0.7)

Z = [[j for j in X] for i in Y]
for ix in range(len(X)):
    for iy in range(len(Y)):
        Z[ix][iy] = m_integrand(guess_N1, guess_N2, guess_N3, mu)(X[ix], Y[iy], [0, 1, 2])[0]

print np.array_str(np.array(Z), precision=5)

In [ ]:
print guess_N1, guess_N2, guess_N3

In [ ]:
#### Recreate Energy Graphs
import matplotlib as mpl
from matplotlib import pyplot
xmax = 3.14
X = np.arange(-xmax, xmax, 0.01)

fig = mpl.pyplot.figure()
zero_statistics()
mus = [-0.031, 0.034, 0.071]
#mus = [0.14, 0.2, 0.22, 0.26]
print global_params_str()

for mu_i in range(len(mus)):
    mu = mus[mu_i]
    N = calc_N(mu)
    guess_N1, guess_N2 = N[0], N[1]

    energy = m_energy(N[0], N[1], N[1], mu)
    integrand = m_integrand(N[0], N[1], N[1], mu)

    energy_sp = fig.add_subplot(1, len(mus), mu_i+1)

    for m in range(3):
        Y = [energy(x, 0, [m])[0] for x in X]
        energy_sp.plot(X, Y, ["r-", "b-", "g-"][m], linewidth = 4.0)
        energy_sp.set_autoscale_on(False)
        
    energy_sp.plot([-xmax, xmax], [mu, mu], "pc--", linewidth = 4.0)
    energy_sp.set_title("mu=%f" % mu)

mpl.pyplot.show(fig)

In [ ]:
guess_N1, guess_N2, guess_N3 = 0.00900515,  0.0039268, 0.0039268

In [ ]:
##### Show Statistics
import matplotlib as mpl
from matplotlib import pyplot

print "The integrand was called", integrand_counter, "times"
print "The energy was called",energy_counter, "times"
print "The integraion included ", bound_counter, "different x values"

dim = 400
Z = np.zeros((dim*2, dim*2))

for p in statistics:
    zx = p[0] * dim / pi
    zy = p[1] * dim / pi
    Z[int(zy)+dim, int(zx)+dim] += 10

pyplot.matshow(Z, extent=(0, pi, 0, pi))
pyplot.show()

In [ ]:
# save cache
import pickle
pickle.dump(cache, open("new_database", "w"))

In [ ]:
# load cache
import pickle
cache = pickle.load(open("new_database", "r"))

In [ ]:
### Create the n's graphs for different Us

mus = [-0.02 + i*0.02 for i in range(15)]
Us = [2.7]

fig = mpl.pyplot.figure()
zero_statistics()
for u in range(len(Us)):
    global U
    U = Us[u]
    
    orbit1, orbit2 = [], []
    for mu in mus:
        guess_N1, guess_N2 = calc_N(mu)
        o1, o2, _ = calc_n(mu, [guess_N1, guess_N2])
        orbit1.append(o1)
        orbit2.append(o2)
        
    ns_sp = fig.add_subplot(1, len(Us), u+1)
    ns_sp.plot(mus, orbit1, "r-", linewidth = 4.0)
    ns_sp.plot(mus, orbit2, "b-", linewidth = 4.0)
    ns_sp.set_title("U = %f" % U)

mpl.pyplot.show()

In [ ]:
print cache.keys()

In [ ]:
cache

In [ ]:
cache.keys() #["3.905000,tl=0.875000,th=0.040000,td=0.040000,U=5.000000,dE=0.047000,dso=0.040000"]

In [ ]:
print len('a=3.905000,tl=0.875000,th=0.040000,td=0.040000,U=5.000000,dE=0.047000,dso=0.040000')
print len("3.905000,tl=0.875000,th=0.040000,td=0.040000,U=5.000000,dE=0.047000,dso=0.040000")

In [ ]:
cache['a=3.905000,tl=0.875000,th=0.040000,td=0.040000,U=5.000000,dE=0.047000,dso=0.040000']

In [ ]:
import numpy as np
from scipy.integrate import odeint

def _infunc(x, func, gfun, hfun, more_args):
    a = gfun(x)
    b = hfun(x)
    y0 = f(x, a)
    return odeint(lambda v, y: f(x, y, *more_args), y0=y0, t=[a, b] )[1]

def dblodeint(f, a, b, gfun, hfun, args=()):
    y0 = f(a, gfun(a), *args)
    return odeint(lambda v, y: _infunc(y, f, gfun, hfun, args),
                  y0=y0, t=[a, b])[1]

if __name__ == '__main__':    
    def f(x, y):
        return np.array([x*y**2, x**2*y, x**4*y, x**6*y], float)

    def exact_int(a, b, ya, yb):
        return np.array([(b**2-a**2)*(yb**3-ya**3)/6.,
                         (b**3-a**3)*(yb**2-ya**2)/6.,
                         (b**5-a**5)*(yb**2-ya**2)/10.,
                         (b**7-a**7)*(yb**2-ya**2)/14.], float)

    print 'approx:', dblodeint(f, 0, 10, lambda x:0, lambda x:10)

    print 'exact:', exact_int(0, 10, 0, 10)


In [ ]:
a = np.array([1 ,2])
b = np.array([2, 3])
a += b*2
a += [43, 12]
print a

In [ ]:
cache = {}